# Finding the best "hygge" path 🏞️

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as cx
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


## Data
### Bike paths
To get the bike paths, we use _OpenStreetMap_ with the library `osmnx`. We query OSM with the _Københavns Kommune_ and _Frederiksberg Kommune_ geocodes. We specify `network_type='bike'` to strictly get bike paths. Since the streets are danish, we use CRS _EPSG:25832_ to work with coordinates in metric units.

In [2]:
geocodes = ['Københavns Kommune', 'Frederiksberg Kommune']
G = ox.graph_from_place(geocodes, network_type='bike')
G = ox.project_graph(G, to_crs='EPSG:25832')

# convert graph to geodataframe
nodes, edges = ox.graph_to_gdfs(G)

/opt/conda/lib/python3.9/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/conda/lib/python3.9/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


### Bird attributes
Bird data is extracted in the [`bird-edges.ipynb`](/notebooks/bird-edges.ipynb) notebook. Birds are attributed to an edge if a 50m buffer around the bird intersects with a given street.

In [8]:
birds = pd.read_csv('../data/bird_edges.csv').set_index(['u', 'v', 'key'])
birds.head()

birds  birds_per_m
u          v          key                    
118725     298795432  0      8.0     0.070877
           6357644306 0      3.0     0.074080
298795432  298795496  0      7.0     0.077950
6357644306 2378383724 0      2.0     0.091199
           6357644350 0      1.0     0.043373

### Tree attributes
![image.png](../pictures/trees.png)

### Traffic noise attributes
![image.png](../pictures/noise.png)

### Putting it all together
Once all the individual data parts are loaded, they are joined into a single dataframe.

In [13]:
gdf = edges.merge(birds, right_index=True, left_index=True, how='left')
... # trees
... # noise

In [14]:
gdf.head()

osmid  oneway              name   highway  \
u          v          key                                                  
118725     298795432  0     27226011    True  Banegårdspladsen  tertiary   
           6357644306 0    140412993    True     Vesterbrogade  tertiary   
298795432  298795496  0    244360867    True  Banegårdspladsen  tertiary   
6357644306 2378383724 0    678991574    True     Vesterbrogade  tertiary   
           6357644350 0    679140293    True     Vesterbrogade  tertiary   

                          maxspeed   length  \
u          v          key                     
118725     298795432  0         30  112.872   
           6357644306 0         50   40.497   
298795432  298795496  0         30   89.801   
6357644306 2378383724 0         50   21.930   
           6357644350 0         50   23.056   

                                                                    geometry  \
u          v          key                                                      
118725     298795432  0    LINESTRING (724025.025 6175551.339, 724031.889...   
           6357644306 0    LINESTRING (724025.025 6175551.339, 724051.956...   
298795432  298795496  0    LINESTRING (724075.369 6175450.104, 724076.732...   
6357644306 2378383724 0    LINESTRING (724059.919 6175572.112, 724075.704...   
           6357644350 0    LINESTRING (724059.919 6175572.112, 724081.508...   

                          lanes bridge  ref service width access tunnel  \
u          v          key                                                 
118725     298795432  0     NaN    NaN  NaN     NaN   NaN    NaN    NaN   
           6357644306 0       2    NaN  NaN     NaN   NaN    NaN    NaN   
298795432  298795496  0     NaN    NaN  NaN     NaN   NaN    NaN    NaN   
6357644306 2378383724 0       2    NaN  NaN     NaN   NaN    NaN    NaN   
           6357644350 0       1    NaN  NaN     NaN   NaN    NaN    NaN   

                          junction  birds  birds_per_m  
u          v          key                               
118725     298795432  0        NaN    8.0     0.070877  
           6357644306 0        NaN    3.0     0.074080  
298795432  298795496  0        NaN    7.0     0.077950  
6357644306 2378383724 0        NaN    2.0     0.091199  
           6357644350 0        NaN    1.0     0.043373

## Finding the best "hygge" path